## Tool Integration

In [1]:
# Advanced Tool Integration - Production Patterns
import asyncio
import json
import time
import requests
import pandas as pd
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional
from dataclasses import dataclass
import logging
from functools import wraps
import sqlite3

# Configure logging for production monitoring
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print(" ADVANCED TOOL INTEGRATION")
print("=" * 35)
print(f"Session: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Focus: Production tool patterns with error handling")
print()

@dataclass
class ToolExecutionResult:
    """Production tool execution tracking"""
    tool_name: str
    success: bool
    result: Any
    execution_time: float
    error_message: Optional[str] = None
    retry_count: int = 0

print("Production tool execution framework initialized")
print("Error tracking, retry logic, performance monitoring")

 ADVANCED TOOL INTEGRATION
Session: 2025-07-26 18:01:46
Focus: Production tool patterns with error handling

Production tool execution framework initialized
Error tracking, retry logic, performance monitoring


## Tool Decorator Integration

Containing

Error Recovery Patterns:
- Retry Logic: Exponential backoff for transient failures
- Circuit Breaker: Fail-fast when services are down
- Fallback Strategies: Alternative approaches when primary tools fail
- Resource Management: Rate limiting and connection pooling

In [2]:
# Production Tool Decorators for Enterprise Reliability

def retry_on_failure(max_retries=3, delay=1.0, backoff=2.0):
    """Enterprise retry decorator with exponential backoff"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            current_delay = delay
            
            for attempt in range(max_retries + 1):
                try:
                    start_time = time.time()
                    result = func(*args, **kwargs)
                    execution_time = time.time() - start_time
                    
                    logger.info(f"{func.__name__} succeeded on attempt {attempt + 1} ({execution_time:.2f}s)")
                    return ToolExecutionResult(
                        tool_name=func.__name__,
                        success=True,
                        result=result,
                        execution_time=execution_time,
                        retry_count=attempt
                    )
                    
                except Exception as e:
                    last_exception = e
                    logger.warning(f"{func.__name__} failed on attempt {attempt + 1}: {e}")
                    
                    if attempt < max_retries:
                        logger.info(f"Retrying in {current_delay:.1f}s...")
                        time.sleep(current_delay)
                        current_delay *= backoff
            
            # All retries failed
            return ToolExecutionResult(
                tool_name=func.__name__,
                success=False,
                result=None,
                execution_time=0.0,
                error_message=str(last_exception),
                retry_count=max_retries
            )
        return wrapper
    return decorator

def rate_limit(calls_per_minute=30):
    """Rate limiting decorator for API tools"""
    call_times = []
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            current_time = time.time()
            
            # Remove calls older than 1 minute
            call_times[:] = [t for t in call_times if current_time - t < 60]
            
            # Check rate limit
            if len(call_times) >= calls_per_minute:
                sleep_time = 60 - (current_time - call_times[0])
                logger.info(f"Rate limit reached for {func.__name__}. Waiting {sleep_time:.1f}s")
                time.sleep(sleep_time)
            
            call_times.append(current_time)
            return func(*args, **kwargs)
        return wrapper
    return decorator

print(" Production tool decorators ready:")
print(" Retry logic with exponential backoff")
print(" Rate limiting for API compliance")
print(" Error tracking and performance monitoring")

 Production tool decorators ready:
 Retry logic with exponential backoff
 Rate limiting for API compliance
 Error tracking and performance monitoring


## Tools Implementation

In [3]:
# Additional import required for file system operations and counters
import os
from collections import defaultdict

# Counter to track tool usage across test runs
tool_usage_counter = defaultdict(int)

@retry_on_failure(max_retries=2, delay=1.5, backoff=2.0)
@rate_limit(calls_per_minute=20)
def process_data_file(file_path: str, operation: str) -> Dict[str, Any]:
    """
    Advanced file processing with multiple format support and error handling.
    """
    tool_usage_counter['process_data_file'] += 1
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    # ... (rest of the function logic is the same)
    file_size = os.path.getsize(file_path)
    if file_size > 50 * 1024 * 1024: raise ValueError(f"File too large: {file_size / 1024 / 1024:.1f}MB")
    file_ext = os.path.splitext(file_path)[1].lower()
    if file_ext == '.csv':
        df = pd.read_csv(file_path)
        if operation == 'analyze': return {'rows': len(df), 'columns': list(df.columns)}
        elif operation == 'summarize': return {'total_rows': len(df), 'numeric_summary': df.describe().to_dict()}
    elif file_ext in ['.xlsx', '.xls']:
        excel_file = pd.ExcelFile(file_path)
        sheets_data = {name: len(pd.read_excel(file_path, sheet_name=name)) for name in excel_file.sheet_names[:3]}
        return {'file_type': 'excel', 'sheets_processed': list(sheets_data.keys())}
    elif file_ext == '.json':
        with open(file_path, 'r', encoding='utf-8') as f: data = json.load(f)
        return {'file_type': 'json', 'structure': type(data).__name__, 'size': len(data) if hasattr(data, '__len__') else 1}
    else:
        raise ValueError(f"Unsupported file type: {file_ext}")


@retry_on_failure(max_retries=3, delay=2.0, backoff=2.5)
@rate_limit(calls_per_minute=15)
def fetch_api_data(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Production API integration with real open-source APIs.
    """
    tool_usage_counter['fetch_api_data'] += 1
    if params is None: params = {}
    TIMEOUT_SECONDS = 10
    if endpoint == 'weather':
        lat = params.get('latitude', -6.2088); lon = params.get('longitude', 106.8456)
        response = requests.get("https://api.open-meteo.com/v1/forecast", params={"latitude": lat, "longitude": lon, "current_weather": "true"}, timeout=TIMEOUT_SECONDS)
        response.raise_for_status(); data = response.json()
        return {"source": "Open-Meteo API", **data['current_weather']}
    elif endpoint == 'crypto_price':
        coin_id = params.get('id', 'bitcoin'); currency = params.get('currency', 'usd')
        response = requests.get(f"https://api.coingecko.com/api/v3/simple/price", params={"ids": coin_id, "vs_currencies": currency, "include_24hr_change": "true"}, timeout=TIMEOUT_SECONDS)
        response.raise_for_status(); data = response.json()
        if coin_id not in data: raise ValueError(f"Cryptocurrency '{coin_id}' not found.")
        return {"source": "CoinGecko API", "coin": coin_id, "currency": currency, **data[coin_id]}
    else:
        raise ValueError(f"Unknown API endpoint: {endpoint}")

@retry_on_failure(max_retries=2)
# CORRECT: Changed the type hint from 'List[Any]' to 'List[str]' for maximum compatibility.
def database_query(query: str, params: Optional[List[str]] = None) -> Dict[str, Any]:
    """
    Production database tool with expanded schema for testing.
    """
    tool_usage_counter['database_query'] += 1
    with sqlite3.connect(':memory:') as conn:
        cursor = conn.cursor()
        # Setup tables for demonstration
        cursor.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, city TEXT)")
        cursor.execute("CREATE TABLE orders (id INTEGER, user_id INTEGER, amount REAL, product TEXT)")
        cursor.executemany("INSERT INTO users VALUES (?, ?, ?)", [(1, 'Alice', 'Jakarta'), (2, 'Bob', 'Surabaya')])
        cursor.executemany("INSERT INTO orders VALUES (?, ?, ?, ?)", [(101, 1, 750.0, 'Laptop'), (102, 2, 25.5, 'Mouse')])
        conn.commit()
        
        cursor.execute(query, params or ())
        columns = [d[0] for d in cursor.description]
        rows = [dict(zip(columns, row)) for row in cursor.fetchall()]
        return {"query": query, "row_count": len(rows), "results": rows}

print("Advanced Tools Updated:")
print("- All tools now increment the `tool_usage_counter`.")
print("- `database_query` now includes `users` and `orders` tables for testing.")

Advanced Tools Updated:
- All tools now increment the `tool_usage_counter`.
- `database_query` now includes `users` and `orders` tables for testing.


## Agent Production using Tool

In [8]:
import os
import json
import sqlite3
import requests
import pandas as pd
import logging
from typing import Dict, Any, Optional, List # Import Optional and List
from collections import defaultdict # Import defaultdict

# --- Setup for Production Patterns ---
# Setup basic logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Counter to track tool usage across test runs
tool_usage_counter = defaultdict(int)


# --- Advanced Tool Implementations ---

# Note: The @retry_on_failure and @rate_limit decorators are assumed to be defined in a previous cell.
# If they are not, you would need to include their definitions here.

@retry_on_failure(max_retries=2)
@rate_limit(calls_per_minute=20)
def process_data_file(file_path: str, operation: str) -> Dict[str, Any]:
    """
    Processes a data file (CSV, Excel, JSON) to perform an operation.
    
    Args:
        file_path (str): The local path to the file.
        operation (str): The operation to perform. Must be 'analyze' or 'summarize'.
    """
    tool_usage_counter['process_data_file'] += 1
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")

    file_size = os.path.getsize(file_path)
    if file_size > 50 * 1024 * 1024:  # 50MB limit
        raise ValueError(f"File too large: {file_size / 1024 / 1024:.1f}MB (limit: 50MB)")

    file_ext = os.path.splitext(file_path)[1].lower()

    try:
        if file_ext == '.csv':
            df = pd.read_csv(file_path)
            if operation == 'analyze':
                return {'rows': len(df), 'columns': list(df.columns)}
            elif operation == 'summarize':
                return {'total_rows': len(df), 'numeric_summary': df.describe().to_dict()}

        elif file_ext in ['.xlsx', '.xls']:
            excel_file = pd.ExcelFile(file_path)
            sheets_data = {name: len(pd.read_excel(file_path, sheet_name=name)) for name in excel_file.sheet_names[:3]}
            return {'file_type': 'excel', 'sheets_processed': list(sheets_data.keys())}

        elif file_ext == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            return {'file_type': 'json', 'structure': type(data).__name__, 'size': len(data) if hasattr(data, '__len__') else 1}

        else:
            raise ValueError(f"Unsupported file type: {file_ext}")

    except Exception as e:
        logger.error(f"File processing error in '{file_path}': {e}")
        raise

@retry_on_failure(max_retries=3, delay=2.0)
@rate_limit(calls_per_minute=15)
def fetch_api_data(endpoint: str, params: Optional[Dict[str, Any]] = None) -> Dict[str, Any]:
    """
    Fetches data from a specified public API endpoint.
    
    Args:
        endpoint (str): The API to call. Must be one of: 'weather', 'crypto_price', 'space_news'.
        params (dict): A dictionary of parameters for the API call.
            - For 'weather', params can include 'latitude' and 'longitude'.
            - For 'crypto_price', params must include 'id' (e.g., 'bitcoin') and 'currency' (e.g., 'usd').
    """
    tool_usage_counter['fetch_api_data'] += 1
    if params is None:
        params = {}

    TIMEOUT_SECONDS = 10

    try:
        if endpoint == 'weather':
            lat = params.get('latitude', -6.2088) # Default to Jakarta
            lon = params.get('longitude', 106.8456)
            response = requests.get("https://api.open-meteo.com/v1/forecast", params={"latitude": lat, "longitude": lon, "current_weather": "true"}, timeout=TIMEOUT_SECONDS)
            response.raise_for_status()
            data = response.json()
            return {"source": "Open-Meteo API", **data['current_weather']}

        elif endpoint == 'crypto_price':
            coin_id = params.get('id', 'bitcoin')
            currency = params.get('currency', 'usd')
            response = requests.get(f"https://api.coingecko.com/api/v3/simple/price", params={"ids": coin_id, "vs_currencies": currency, "include_24hr_change": "true"}, timeout=TIMEOUT_SECONDS)
            response.raise_for_status()
            data = response.json()
            if coin_id not in data:
                raise ValueError(f"Cryptocurrency '{coin_id}' not found.")
            return {"source": "CoinGecko API", "coin": coin_id, "currency": currency, **data[coin_id]}

        elif endpoint == 'space_news':
            api_url = "https://api.spaceflightnewsapi.net/v4/articles/"
            limit = params.get('limit', 3)
            response = requests.get(api_url, params={"limit": limit}, timeout=TIMEOUT_SECONDS)
            response.raise_for_status()
            return {"source": "Spaceflight News API", "count": len(response.json()['results']), "articles": response.json()['results']}
            
        else:
            # This will now correctly raise an error for hallucinated endpoints.
            raise ValueError(f"Unknown API endpoint: '{endpoint}'. Please use 'weather', 'crypto_price', or 'space_news'.")

    except requests.exceptions.RequestException as e:
        logger.error(f"API request failed for endpoint '{endpoint}': {e}")
        raise ConnectionError(f"Failed to connect to API endpoint '{endpoint}'.") from e

@retry_on_failure(max_retries=2)
# CORRECT: Changed the type hint from 'List[Any]' to 'List[str]' for maximum compatibility.
def database_query(query: str, params: Optional[List[str]] = None) -> Dict[str, Any]:
    """
    Executes a read-only SQL SELECT query against an in-memory database.
    The database contains two tables:
    1. users(id INTEGER, name TEXT, city TEXT)
    2. orders(id INTEGER, user_id INTEGER, amount REAL, product TEXT)
    
    Args:
        query (str): The SQL SELECT statement to execute.
        params (list): A list of parameters (as strings) to safely bind to the query.
    """
    tool_usage_counter['database_query'] += 1
    # Only allow SELECT statements for security
    if not query.strip().lower().startswith('select'):
        raise ValueError("Only SELECT queries are allowed.")
        
    with sqlite3.connect(':memory:') as conn:
        cursor = conn.cursor()
        # Setup tables and data for demonstration
        cursor.execute("CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, city TEXT)")
        cursor.execute("CREATE TABLE orders (id INTEGER, user_id INTEGER, amount REAL, product TEXT)")
        cursor.executemany("INSERT INTO users VALUES (?, ?, ?)", [(1, 'Alice', 'Jakarta'), (2, 'Bob', 'Surabaya')])
        cursor.executemany("INSERT INTO orders VALUES (?, ?, ?, ?)", [(101, 1, 750.0, 'Laptop'), (102, 2, 25.5, 'Mouse'), (103, 1, 550.0, 'Keyboard')])
        conn.commit()
        
        cursor.execute(query, params or [])
        columns = [d[0] for d in cursor.description]
        rows = [dict(zip(columns, row)) for row in cursor.fetchall()]
        return {"query": query, "row_count": len(rows), "results": rows}


print("\nAdvanced Tools Ready:")
print("   File processing: CSV, Excel, JSON with error handling")
print("   API integration: Real-time Weather, Crypto Prices, and Space News with retry logic")
print("   Database tools: Safe queries with connection management")
print("   Production patterns: Rate limiting, monitoring, validation")


Advanced Tools Ready:
   File processing: CSV, Excel, JSON with error handling
   API integration: Real-time Weather, Crypto Prices, and Space News with retry logic
   Database tools: Safe queries with connection management
   Production patterns: Rate limiting, monitoring, validation


In [9]:
# Correct imports for the Google Agent Development Kit
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # For multi-model support
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner as agent_runner
from google.genai import types # For creating message Content/Parts

# --- Agent Setup ---
# Initialize the ADK components
try:
    # 1. Define the tools
    all_tools = [process_data_file, fetch_api_data, database_query]

    # 2. Instantiate the LLM model
    llm = LiteLlm(model="ollama_chat/llama3.1:8b")

    # 3. Instantiate the Agent with a name, tools, and the model
    agent = Agent(name="tool_agent", model=llm, tools=all_tools)

    # 4. Instantiate the Session Service
    session_service = InMemorySessionService()

    # 5. Instantiate the Runner with all required components
    runner = agent_runner(app_name="tool_app", agent=agent, session_service=session_service)
    
    print("ADK Agent and Runner initialized successfully.")

except Exception as e:
    print(f"Failed to initialize ADK components: {e}")
    print("   Please ensure you have the Google ADK installed (`pip install google-adk`).")
    runner = None # Set runner to None to avoid errors in the next step

# --- Production Tool Demonstration ---

async def test_advanced_tools():
    """Run a suite of tests on an ADK-based agent with tool integration."""
    if not runner:
        print("\nSkipping tests because ADK components are not available.")
        return

    # CORRECT: Provide all required arguments to create the session.
    await runner.session_service.create_session(
        session_id="main", app_name="tool_app", user_id="system"
    )

    # Test requests that trigger the tools
    test_requests = [
        "Get the current weather in Jakarta",
        "Search the orders table where amount > 500",
        "Get weather for Bukittinggi, then get crypto info for ethereum and solana",
        "Find all orders in the database and get current weather in Palembang"
    ]

    print("\nADVANCED TOOL INTEGRATION TESTING")
    print("=" * 42)

    for i, request in enumerate(test_requests, 1):
        print(f"\nTest {i}: {request}")
        print("-" * 50)
        message = types.Content(role="user", parts=[types.Part(text=request)])
        response_text = "Error: Agent did not produce a final response."

        try:
            async for event in runner.run_async(user_id="system", session_id="main", new_message=message):
                if event.is_final_response():
                    response_text = event.content.parts[0].text
                    break
        except Exception as e:
            response_text = f"An error occurred during agent execution: {e}"

        print(f"Agent Response:\n{response_text}")
        await asyncio.sleep(1) # Avoid overwhelming APIs

# --- Execution and Analysis ---

# Run the tool demonstration
await test_advanced_tools()

print("\nTOOL INTEGRATION ANALYSIS")
print("=" * 35)
print("Tool Invocation Notes:")
print("   - Tools are triggered implicitly by the LLM based on user prompts.")
print("   - ADK handles the mapping from LLM output to tool calls.")
print("   - The print logs and counters inside each tool confirm their execution.")

if any(tool_usage_counter.values()):
    print("\nTOOL USAGE SUMMARY")
    for tool, count in sorted(tool_usage_counter.items()):
        print(f"   - {tool}: {count} call(s)")
else:
    print("\nTOOL USAGE SUMMARY")
    print("No tools were called. Check agent logs for details.")

print("\nADVANCED TOOL INTEGRATION COMPLETE")
print("- Agent has processed a suite of tool-based tasks.")

21:59:57 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat


ADK Agent and Runner initialized successfully.

ADVANCED TOOL INTEGRATION TESTING

Test 1: Get the current weather in Jakarta
--------------------------------------------------


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:__main__:fetch_api_data succeeded on attempt 1 (1.00s)
22:03:55 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http:/

Agent Response:
The current weather in Jakarta is:

* Temperature: 27.6°C
* Wind Speed: 2.8 m/s
* Wind Direction: 140°
* It's night time.

Please note that the weather data may have changed since the API call was made. For up-to-date information, please check with a reliable source.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
22:05:26 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat


...


Test 4: Find all orders in the database and get current weather in Palembang
--------------------------------------------------


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:__main__:Retrying in 1.0s...
INFO:__main__:Retrying in 2.0s...
INFO:__main__:fetch_api_data succeeded on attempt 1 (0.86s)
22:19:23 - LiteLLM:INFO: utils.py:3230 - 
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:LiteLLM:
LiteLLM completion() model= llama3.1:8b; provider = ollama_chat
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:1

Agent Response:
Here is the information you requested:

**Orders in the database:**

Unfortunately, there are no orders in the database.

**Current Weather in Palembang:**

* Temperature: 24.5°C
* Wind Speed: 3.4 m/s
* Wind Direction: 72°
* It's night time.

Please note that the weather data may have changed since the API call was made. For up-to-date information, please check with a reliable source.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/show "HTTP/1.1 200 OK"



TOOL INTEGRATION ANALYSIS
Tool Invocation Notes:
   - Tools are triggered implicitly by the LLM based on user prompts.
   - ADK handles the mapping from LLM output to tool calls.
   - The print logs and counters inside each tool confirm their execution.

TOOL USAGE SUMMARY
   - database_query: 4 call(s)
   - fetch_api_data: 5 call(s)

ADVANCED TOOL INTEGRATION COMPLETE
- Agent has processed a suite of tool-based tasks.
